In [49]:
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re

In [50]:
GENIUS_API_TOKEN='78CERPnMulsX6Hb7Gvbw8ESZx1h4gT1mIURMeydwlMxpOfWD5LtJTBFEixQE_4OP'

# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
request_song_url('Manal', 1)

Found 1 songs by Manal


['https://genius.com/Manal-slay-lyrics']

In [51]:
import requests
from bs4 import BeautifulSoup
import re
import os
from nltk.tokenize import word_tokenize

# Function to scrape lyrics from Genius
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    
    # Find the lyrics container using a class that contains "Lyrics__Container"
    lyrics_divs = html.select("div[class*=Lyrics__Container]")
    
    # Extract text from all matching divs and join them into one string
    lyrics = '\n'.join([div.get_text() for div in lyrics_divs])
    
    # Removing unnecessary identifiers like chorus, verse, etc.
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)  # Remove text inside parentheses or square brackets
    
    # Remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    
    return lyrics

# Function to normalize spacing
def normalize_spacing(text):
    # Add space before capital letters if not preceded by a space
    text = re.sub(r'(?<!\s)([A-Z])', r' \1', text)
    # Add space after punctuation if missing
    text = re.sub(r'([?!.,:;])(?=\S)', r'\1 ', text)
    # Normalize multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Function to save lyrics to a file
def save_lyrics_to_file(url, dir, filename="song1.txt"):
    # Ensure the directory exists
    os.makedirs(dir, exist_ok=True)
    
    # Construct the full path
    file_path = os.path.join(dir, filename)
    
    # Get the lyrics
    lyrics = scrape_song_lyrics(url)
    
    # Normalize the lyrics
    normalized_lyrics = normalize_spacing(lyrics)
    
    # Save the lyrics to the specified file path
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(normalized_lyrics)

# URLs of the songs
urls = [
    ('https://genius.com/Manal-slay-lyrics', 'song1.txt'),
    ('https://genius.com/elgrandetoto-mghayer-lyrics', 'song2.txt'),
    ('https://genius.com/stormy-africain-lyrics', 'song3.txt')
]

# Directory to save lyrics
dir = "C:/Users/ikry/NLP-darija/data"

# Download, normalize, and save lyrics for each song
for url, filename in urls:
    save_lyrics_to_file(url, dir, filename)
